In [90]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET

In [140]:
buy_out = df(columns=["date",
                        "phone_number",
                        "notebook",
                        "tablet",
                        "phone",
                        "spare_part",
                        "conversation",
                        "customer_price",
                        "our_price",
                        "recycle",
                        "customer_decision"])

map_facts_to_columns = dict()
map_facts_to_columns["phone_number"] = ["CustomerPhone"]
map_facts_to_columns["notebook"] = ["Word", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["phone"] = ["Phone", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["tablet"] = ["Tablet", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["conversation"] = ["Buy"]
map_facts_to_columns["recycle"] = ["source sentence"]
map_facts_to_columns["customer_decision"] = ["what_customer_expects"]

buy_out

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [136]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;

make_dict_of_leads()["46"]

'89140580517 продажа айфон6s на выезде Ляпидевского 12.'

In [154]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

calls = df(columns=[
        "CustomerPhone",
        "FromUU",
        "Buy",
        "Pawn",
        "Notebook_Word",
        "Notebook_Vendor",
        "Notebook_Model",
        "Notebook_UserDefinedName",
        "Phone_Word",
        "Phone_Vendor",
        "Phone_Model",
        "Phone_UserDefinedName",
        "source_sentence"
    ])

facts = compare_facts_to_leads()
leads = make_dict_of_leads()
for lead in facts:
    elems = facts[lead]
    one_sentence = lead
    for fact_name in elems:
        for fact_field in fact_name:
            one_sentence += " | >" + i.attrib["val"] + "<"
        one_sentence += " | " + e.tag
        if e.tag.lower() == "BUY".lower():
            print(i.tag, one_sentence)
            print(lead, leads[lead])
    

            

NumberInPrice 10 | >BUY< | >17< | Buy
10 89140580517 куп 17 зац.
Word 3 | >89140580517< | CustomerPhone | >УУ< | CustomerPlace | >НОУТБУК< | Notebook | >BUY< | Buy
3 89140580517 уу ноутбук купить за 17000 руб.
Word 55 | >89140580517< | CustomerPhone | >BUY< | Buy
55 89140580517 жесткий диск купить 320.
Word 6 | >89140580517< | CustomerPhone | >BUY< | Buy
6 89140580517 куп тел 27.
Word 7 | >89140580517< | CustomerPhone | >BUY< | Buy
7 89140580517 куп телефон 27.
NumberInPrice 16 | >BUY< | >17< | Buy
16 89140580517 buy 17 зац.
NumberInPrice 9 | >BUY< | >12< | Buy
9 89140580517 куп 12 выезд Широких-Полянского 27, Виталий.
NumberInPrice 5 | >BUY< | >12< | Buy
5 89140580517 куп 12.
NumberInPrice 12 | >BUY< | >17< | Buy
12 89140580517 покупка 17 зац.
NumberInPrice 17 | >BUY< | >17< | Buy
17 89140580517 b 17 зац.
Word 1 | >89140580517< | CustomerPhone | >BUY< | Buy
1 89140580517 хочет купить ноутбук за 17000 руб ездить Якутия.
Word 4 | >89140580517< | CustomerPhone | >НОУТБУК< | Notebook | >У

,CustomerPhone,FromUU,Buy,Pawn,Notebook_Word,Notebook_Vendor,Notebook_Model,Notebook_UserDefinedName,Phone_Word,Phone_Vendor,Phone_Model,Phone_UserDefinedName,source_sentence
